## Investigating Systematic Trading Strategies

In this script, we will utilize Python in order to backtest the Dragon Fly DOJI candle formation in locating market bottoms

Since we will be backtesting numerous different stocks, we will use the Yahoo Finance API to gather the OHLC data. 

In [3]:
!pip install yfinance

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 608.1 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 499.4/499.4 kB 1.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.2/112.2 kB 1.9 MB/s eta 0:00:00a 0:00:01
  Using cached multitasking-0.0.11-py3-none-any.whl (8.5 kB)
  Attempting uninstall: pytz
    Found existing installation: pytz 2022.1
    Uninstalling pytz-2022.1:
      Successfully uninstalled pytz-2022.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
conda-repo-cli 1.0.20 requires clyent==1.2.1, but you have clyent 1.2.2 which is incompatible.
conda-repo-cli 1.0.20 requires nbformat==5.4.0, but you have nbformat 5.5.0 which is incompatible.


We will also be using Pandas to frame our historical price data. After importing, we will initialize our criteria for what we consider a loss or a win. For this backtest, we will use 5% (0.05) as the guage. 

In [2]:
from decimal import DivisionByZero
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
lossPercent = 0.05
successsPercent = 0.05

We have selected a diversified list of stocks to get a better idea of the capabilities of the strategy on the market. To do this, we will be testing the strategy on the sector ETFs including the S&P 500 ETF and the Nasdaq 100 ETF. If you would like to backtest on the entire S&P 500, you can use pandas to read the wikipedia list of ticker symbols and add it to a list. For this case however, we will stick to the the ETFs. We will also initialize our win counter and total trades counter. 

In [3]:
# tickers = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0].Symbol.to_list()
# tickers = [i.replace('.', '-') for i in tickers]


tickers = ['SPY', 'QQQ', 'XLE', 'XLF', 'XLV', 'XLU', 'XLI', 'XLP', 'XOP']

totalWin = 0
totalTotal = 0 


To grab our OHLC data, we will call each stock in the list of tickers create a position variable (to know if the stock is in a position or not), an enterPrice variable, an enterTime variable, and the local win and total. Then, we will grab each ticker's historical data from the past 360 days with a timeframe of 60 mins. The structure of a Dragon Fly DOJI candle is a doji candle below the previous candle close with an excetionally long tail. To recreate this structure, we will need the closing price, the previous candle close, the low price, and the open price. After grabbing those price points by looping through the initial dataframe, we then set our conditions for the 'buy'. If bought, the position will be set to entered and it will wait for it to change back to none to find another trade. To be set back to none, it will wait for the sell conditions to hit (5% threshold is met). If the 5% move is in the upside, it will count it as a win and vice verse for the 5% move being to the downside. 

In [4]:
for stock in tickers:

    #start loop for each day
    pos = "none"
    enterPrice = None
    enterTime = None
    win = 0
    total = 0

    enterTimes = []
    enterPrices = []

    hourDf = yf.download(stock, period="360d",interval = "60m")

    hourData = hourDf["Close"]
    closePrv = hourDf["Close"].shift()

    longData = hourDf['Close'].shift(3)


    
    for hour in hourData.index:
        price = hourData[hour]
        closePast = closePrv[hour]
        open = hourDf['Open'][hour]
        high = hourDf['High'][hour]
        low = hourDf['Low'][hour]
        long_data = longData[hour]

        if pos == 'none':
            if open * 1.0035 < closePast:
                if price > open:
                    if price < closePast:
                        if price * 1.01 < long_data:
                            if low * 1.01 < price:
                                enterPrice = price
                                pos = 'entered'


        elif pos == 'entered':
            if price > enterPrice * (1 + successsPercent):
                win += 1
                total += 1
                pos = 'none'
                totalWin += 1
                totalTotal += 1
            elif price < enterPrice * (1 - lossPercent):
                total += 1
                pos = 'none'
                totalTotal += 1
    try: 
        winrate = (win/total) * 100 
    except:
        winrate = 'NA'
    print(f'{stock}: {win}/{total} - {winrate}')



[*********************100%***********************]  1 of 1 completed
SPY: 4/4 - 100.0
[*********************100%***********************]  1 of 1 completed
QQQ: 4/4 - 100.0
[*********************100%***********************]  1 of 1 completed
XLE: 5/9 - 55.55555555555556
[*********************100%***********************]  1 of 1 completed
XLF: 2/3 - 66.66666666666666
[*********************100%***********************]  1 of 1 completed
XLV: 0/0 - NA
[*********************100%***********************]  1 of 1 completed
XLU: 2/2 - 100.0
[*********************100%***********************]  1 of 1 completed
XLI: 1/1 - 100.0
[*********************100%***********************]  1 of 1 completed
XLP: 0/0 - NA
[*********************100%***********************]  1 of 1 completed
XOP: 7/14 - 50.0


To find the overall success rate, we will use the total win and total trades counter. 

In [5]:
winPerc = (totalWin / totalTotal) * 100
print(f'Total: {totalWin}/{totalTotal} - {winPerc}%')

Total: 25/37 - 67.56756756756756%


# # Important Note

Success rates are typically a good determinant in seeing if a particular strategy is profitable in the long term. In this experiment, we developed a strategy that is 67% successful in over 35 trades with a time frame of 1 year. With a quick glance, we can see that the strategy appears to be profitable. HOWEVER, I believe that these results do not, and should not determine if the strategy works consistantly in the long run. If we run the same strategy but for 720 days instead, we see different results. 

In [8]:
# tickers = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0].Symbol.to_list()
# tickers = [i.replace('.', '-') for i in tickers]


tickers = ['SPY', 'QQQ', 'XLE', 'XLF', 'XLV', 'XLU', 'XLI', 'XLP', 'XOP']

totalWin = 0
totalTotal = 0 


In [9]:
for stock in tickers:

    #start loop for each day
    pos = "none"
    enterPrice = None
    enterTime = None
    win = 0
    total = 0

    enterTimes = []
    enterPrices = []

    hourDf = yf.download(stock, period="720d",interval = "60m")

    hourData = hourDf["Close"]
    closePrv = hourDf["Close"].shift()

    longData = hourDf['Close'].shift(3)


    
    for hour in hourData.index:
        price = hourData[hour]
        closePast = closePrv[hour]
        open = hourDf['Open'][hour]
        high = hourDf['High'][hour]
        low = hourDf['Low'][hour]
        long_data = longData[hour]

        if pos == 'none':
            if open * 1.0035 < closePast:
                if price > open:
                    if price < closePast:
                        if price * 1.01 < long_data:
                            if low * 1.01 < price:
                                enterPrice = price
                                pos = 'entered'


        elif pos == 'entered':
            if price > enterPrice * (1 + successsPercent):
                win += 1
                total += 1
                pos = 'none'
                totalWin += 1
                totalTotal += 1
            elif price < enterPrice * (1 - lossPercent):
                total += 1
                pos = 'none'
                totalTotal += 1
    try: 
        winrate = (win/total) * 100 
    except:
        winrate = 'NA'
    print(f'{stock}: {win}/{total} - {winrate}')



[*********************100%***********************]  1 of 1 completed
SPY: 6/8 - 75.0
[*********************100%***********************]  1 of 1 completed
QQQ: 7/11 - 63.63636363636363
[*********************100%***********************]  1 of 1 completed
XLE: 10/20 - 50.0
[*********************100%***********************]  1 of 1 completed
XLF: 4/9 - 44.44444444444444
[*********************100%***********************]  1 of 1 completed
XLV: 0/2 - 0.0
[*********************100%***********************]  1 of 1 completed
XLU: 2/5 - 40.0
[*********************100%***********************]  1 of 1 completed
XLI: 2/4 - 50.0
[*********************100%***********************]  1 of 1 completed
XLP: 3/4 - 75.0
[*********************100%***********************]  1 of 1 completed
XOP: 14/28 - 50.0


In [10]:
winPerc = (totalWin / totalTotal) * 100
print(f'Total: {totalWin}/{totalTotal} - {winPerc}%')

Total: 48/91 - 52.74725274725275%


It is clear that over the past 2 years, this strategy is barely more successful than a fair coin flip. The market conditions are a very important factor when testing a strategy. One strategy may work for a certain market condition but not a different one. It is clear from this that even though this strategy worked decently well with an r-multiple of 1:1 in the first 360d, when averaged out with the prior year, it worsens significantly. 

## Conclusion

In this test, we attempted to backtest the Dragon Fly DOJI candle formation to locate market bottoms and our results were inconclusive. 